In [42]:
# Installer les packages si nécessaire
# !pip install pandas psycopg2-binary sqlalchemy python-dotenv

import pandas as pd
import psycopg2
from sqlalchemy import create_engine, text
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports chargés")

✅ Imports chargés


In [43]:
# PostgreSQL Local (Docker)
LOCAL_HOST = "localhost"
LOCAL_PORT = "2345"
LOCAL_DATABASE = "ada"
LOCAL_USER = "human"
LOCAL_PASSWORD = "pass"

# PostgreSQL Azure FlexibleServer
AZURE_HOST = "carter-cash-serveur-postgresql.postgres.database.azure.com"
AZURE_PORT = "5432"
AZURE_DATABASE = "carter_cash_data"  # Base par défaut, ou créer une DB spécifique
AZURE_USER = "carter_cash_serveur_postgresql"
AZURE_PASSWORD = "20eCoVHjWXMTUUdC"

print("✅ Paramètres définis")

✅ Paramètres définis


In [44]:
# Test connexion PostgreSQL Local
try:
    conn_local = psycopg2.connect(
        host=LOCAL_HOST, port=LOCAL_PORT, 
        dbname=LOCAL_DATABASE, user=LOCAL_USER, password=LOCAL_PASSWORD
    )
    print("✅ Connexion PostgreSQL local OK")
    conn_local.close()
except Exception as e:
    print("⛔ Connexion PostgreSQL local échouée :", e)

# Test connexion PostgreSQL Azure
try:
    conn_azure = psycopg2.connect(
        host=AZURE_HOST, port=AZURE_PORT,
        dbname=AZURE_DATABASE, user=AZURE_USER, password=AZURE_PASSWORD,
        sslmode='require'  # Obligatoire pour Azure
    )
    print("✅ Connexion PostgreSQL Azure OK")
    conn_azure.close()
except Exception as e:
    print("⛔ Connexion PostgreSQL Azure échouée :", e)

✅ Connexion PostgreSQL local OK
✅ Connexion PostgreSQL Azure OK


In [45]:
from sqlalchemy import create_engine, text
import pandas as pd

# Connexion locale avec SQLAlchemy
local_engine = create_engine(f"postgresql+psycopg2://{LOCAL_USER}:{LOCAL_PASSWORD}@{LOCAL_HOST}:{LOCAL_PORT}/{LOCAL_DATABASE}")

# Tables spécifiques à migrer (tes données métier)
tables_to_migrate = ["USER_API", "DimensionsParModel", "Produit", "Caracteristiques", "Dimensions"]

# Extraction des données avec text()
local_data = {}

with local_engine.connect() as conn:
    for table in tables_to_migrate:
        try:
            # Utiliser text() pour encapsuler la requête
            query = text(f'SELECT * FROM "{table}"')
            df = pd.read_sql(query, conn)
            local_data[table] = df
            print(f"✅ {table}: {len(df)} lignes extraites")
        except Exception as e:
            print(f"⚠️ Erreur lors de l'extraction de {table}: {e}")

print(f"📊 Total des tables extraites: {len(local_data)}")

✅ USER_API: 3 lignes extraites
✅ DimensionsParModel: 191112 lignes extraites
✅ Produit: 11442 lignes extraites
✅ Caracteristiques: 11442 lignes extraites
✅ Dimensions: 11180 lignes extraites
📊 Total des tables extraites: 5


In [46]:
# Créer une base dédiée sur Azure (optionnel - sinon utiliser 'postgres')
DB_NAME = "carter_cash_data"

try:
    # Connexion admin pour créer la DB
    conn_admin = psycopg2.connect(
        host=AZURE_HOST, port=AZURE_PORT,
        dbname="postgres", user=AZURE_USER, password=AZURE_PASSWORD,
        sslmode='require'
    )
    conn_admin.autocommit = True
    cursor = conn_admin.cursor()
    
    # Créer la base si elle n'existe pas
    cursor.execute(f"CREATE DATABASE {DB_NAME};")
    print(f"✅ Base de données '{DB_NAME}' créée sur Azure")
    
    conn_admin.close()
except psycopg2.errors.DuplicateDatabase:
    print(f"ℹ️ Base de données '{DB_NAME}' existe déjà")
except Exception as e:
    print(f"⚠️ Erreur création DB: {e}")
    DB_NAME = "postgres"  # Utiliser la DB par défaut
    print(f"→ Utilisation de la base par défaut: {DB_NAME}")

ℹ️ Base de données 'carter_cash_data' existe déjà


In [47]:
# Connexion à la base cible sur Azure
azure_engine = create_engine(f"postgresql+psycopg2://{AZURE_USER}:{AZURE_PASSWORD}@{AZURE_HOST}:{AZURE_PORT}/{DB_NAME}?sslmode=require")

# Scripts de création des tables
create_tables_sql = [
    # Table Produit
    '''
    DROP TABLE IF EXISTS "Produit" CASCADE;
    CREATE TABLE "Produit" (
        "ID_Produit" SERIAL PRIMARY KEY,
        "URL_Produit" VARCHAR(500),
        "Prix" INTEGER,
        "Info_generale" VARCHAR(500),
        "Descriptif" VARCHAR(500),
        "Note" VARCHAR(50),
        "Date_scrap" DATE,
        "Marque" VARCHAR(200)
    );
    ''',
    
    # Table USER_API
    '''
    DROP TABLE IF EXISTS "USER_API" CASCADE;
    CREATE TABLE "USER_API" (
        "ID_USER_API" SERIAL PRIMARY KEY,
        "username" VARCHAR(50),
        "email" VARCHAR(150),
        "full_name" VARCHAR(50),
        "hashed_password" VARCHAR(200),
        "Date_Création" DATE,
        "Date_Derniere_Connexion" DATE
    );
    ''',
    
    # Table DimensionsParModel
    '''
    DROP TABLE IF EXISTS "DimensionsParModel" CASCADE;
    CREATE TABLE "DimensionsParModel" (
        "ID_DimensionModel" SERIAL PRIMARY KEY,
        "Marque" VARCHAR(50),
        "Modele" VARCHAR(50),
        "Annee" INTEGER,
        "Finition" VARCHAR(250),
        "Largeur" INTEGER,
        "Hauteur" INTEGER,
        "Diametre" INTEGER
    );
    ''',
    
    # Table Caracteristiques
    '''
    DROP TABLE IF EXISTS "Caracteristiques" CASCADE;
    CREATE TABLE "Caracteristiques" (
        "ID_Caracteristique" SERIAL PRIMARY KEY,
        "Consommation" CHAR(1),
        "Indice_Pluie" CHAR(1),
        "Bruit" INTEGER,
        "Saisonalite" VARCHAR(50),
        "Type_Vehicule" VARCHAR(50),
        "Runflat" VARCHAR(50),
        "ID_Produit" INTEGER REFERENCES "Produit"("ID_Produit")
    );
    ''',
    
    # Table Dimensions
    '''
    DROP TABLE IF EXISTS "Dimensions" CASCADE;
    CREATE TABLE "Dimensions" (
        "ID_Dimension" SERIAL PRIMARY KEY,
        "Largeur" INTEGER,
        "Hauteur" INTEGER,
        "Diametre" INTEGER,
        "Charge" INTEGER,
        "Vitesse" CHAR(1),
        "ID_Produit" INTEGER REFERENCES "Produit"("ID_Produit")
    );
    '''
]

# Exécution des scripts de création
with azure_engine.connect() as conn:
    for sql in create_tables_sql:
        conn.execute(text(sql))
        conn.commit()

print("✅ Tables créées sur Azure PostgreSQL")

✅ Tables créées sur Azure PostgreSQL


In [48]:
# Migration dans l'ordre (tables sans FK d'abord)
migration_order = ["Produit", "USER_API", "DimensionsParModel", "Caracteristiques", "Dimensions"]

for table_name in migration_order:
    if table_name in local_data:
        try:
            df = local_data[table_name]
            
            # Insertion dans Azure PostgreSQL
            df.to_sql(
                table_name, 
                azure_engine, 
                if_exists='append', 
                index=False,
                method='multi'  # Plus rapide pour gros volumes
            )
            
            print(f"✅ {table_name}: {len(df)} lignes migrées vers Azure")
            
        except Exception as e:
            print(f"⛔ Erreur migration {table_name}: {e}")

print("🚀 Migration terminée !")

✅ Produit: 11442 lignes migrées vers Azure
✅ USER_API: 3 lignes migrées vers Azure
✅ DimensionsParModel: 191112 lignes migrées vers Azure
✅ Caracteristiques: 11442 lignes migrées vers Azure
✅ Dimensions: 11180 lignes migrées vers Azure
🚀 Migration terminée !


In [49]:
# Vérification du nombre de lignes dans chaque table sur Azure
verification_results = {}

with azure_engine.connect() as conn:
    for table_name in migration_order:
        try:
            result = conn.execute(text(f'SELECT COUNT(*) FROM "{table_name}"'))
            count = result.fetchone()[0]
            verification_results[table_name] = count
            print(f"📊 {table_name}: {count} lignes sur Azure")
        except Exception as e:
            print(f"⚠️ Erreur vérification {table_name}: {e}")

print("\n" + "="*50)
print("RÉSUMÉ DE LA MIGRATION")
print("="*50)

for table_name in migration_order:
    if table_name in local_data and table_name in verification_results:
        local_count = len(local_data[table_name])
        azure_count = verification_results[table_name]
        status = "✅ OK" if local_count == azure_count else "⚠️ DIFFÉRENCE"
        print(f"{table_name}: Local={local_count} → Azure={azure_count} {status}")

📊 Produit: 11442 lignes sur Azure
📊 USER_API: 3 lignes sur Azure
📊 DimensionsParModel: 191112 lignes sur Azure
📊 Caracteristiques: 11442 lignes sur Azure
📊 Dimensions: 11180 lignes sur Azure

RÉSUMÉ DE LA MIGRATION
Produit: Local=11442 → Azure=11442 ✅ OK
USER_API: Local=3 → Azure=3 ✅ OK
DimensionsParModel: Local=191112 → Azure=191112 ✅ OK
Caracteristiques: Local=11442 → Azure=11442 ✅ OK
Dimensions: Local=11180 → Azure=11180 ✅ OK
